# Interactive question answering with OpenVINO

Goal of this task is to complete a demo that showcases interactive question answering with OpenVINO. We will use [small BERT-large-like model](https://github.com/openvinotoolkit/open_model_zoo/tree/master/models/intel/bert-small-uncased-whole-word-masking-squad-int8-0002) distilled and quantized to INT8 on SQuAD v1.1 training set from larger BERT-large model. The model comes from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/).

At the bottom of this notebook, you will see live inference results from your inputs.


## Imports


In [ ]:
import time

import numpy as np
from openvino.runtime import Core, Dimension

import tokens_bert as tokens
from data_processing import prepare_input, postprocess, load_context


## The model

### Download the model

We use `omz_downloader`, which is a command-line tool from the `openvino-dev` package. `omz_downloader` automatically creates a directory structure and downloads the selected model. These models are already converted to OpenVINO Intermediate Representation (IR) - if they weren't, we'd need to use `omz_converter` first.


In [ ]:
# directory where model will be downloaded
output_model_dir = "model"
cache_model_dir = "model"

# desired precision
precision = "FP16-INT8"

# model name as named in Open Model Zoo
model_name = "bert-small-uncased-whole-word-masking-squad-int8-0002"

model_path = f"model/intel/{model_name}/{precision}/{model_name}.xml"
model_weights_path = f"model/intel/{model_name}/{precision}/{model_name}.bin"
vocab_file_path = "data/vocab.txt"

# create dictionary with words and their indices
vocab = tokens.load_vocab_file(vocab_file_path)


In [ ]:
download_command = f"omz_downloader " \
                   f"--name {model_name} " \
                   f"--precision {precision} " \
                   f"--output_dir {output_model_dir} " \
                   f"--cache_dir {cache_model_dir}"
! $download_command


### Load the model

Downloaded models are located in a fixed structure, which indicates vendor, model name and precision. Only a few lines of code are required to run the model:

- First, we create an Inference Engine object
- Then we read the network architecture and model weights from the .xml and .bin files
- Next, we want to change input shapes so that length of text (on each layer) is no longer static
- Finally, we compile the network for the desired device. OpenVINO currently supports dynamic shapes only with `CPU`.

Replace `pass` with correct code


In [ ]:
# initialize inference engine
ie_core = Core()

# read the model and corresponding weights from file
model = ie_core.read_model(model=model_path, weights=model_weights_path)

# assign dynamic shapes to every input layer
# shape for each layer is [1, 384], where 1 stands for batch size, 384 stands for length
# we need to replace length of text for each layer to be dynamic

for input_layer in model.inputs:
    pass

# compile the model for the CPU
compiled_model = ie_core.compile_model(model=model, device_name="CPU")


input_keys = list(compiled_model.inputs)
output_keys = list(compiled_model.outputs)

Input keys are the names of the input nodes and output keys contain names of output nodes of the network. In the case of the BERT-large-like model, we have four inputs and two outputs.

Replace `None` and `____` with correct code


In [ ]:

[i.any_name for i in input_keys], [o.any_name for o in output_keys]


Now, the data processing happens. You can find it in `data_processing.py`.

Next, it's time to create inference:

- Firstly, we need to create a list of tokens from the context and the question
- Then, we are looking for the best answer in the context. The best answer should come with the highest score.

Replace `None` and `____` with correct code.


In [ ]:
def get_best_answer(question, context, vocab, input_keys):
    # convert context string to tokens
    context_tokens, context_tokens_start_end = tokens.text_to_tokens(
        text=context.lower(), vocab=vocab
    )
    # convert question string to tokens
    question_tokens, _ = tokens.text_to_tokens(text=question.lower(), vocab=vocab)

    network_input = prepare_input(question_tokens, context_tokens, input_keys)
    input_size = len(context_tokens) + len(question_tokens) + 3
    
    # Create inference request
    request = None

    # Run inference using network_input
    request.____

    # Get output tensors from inference in numpy format
    output_start = request.____
    output_end = request.____

    # postprocess the result getting the score and context range for the answer
    score_start_end = postprocess(output_start=output_start,
                                  output_end=output_end,
                                  question_tokens=question_tokens,
                                  context_tokens_start_end=context_tokens_start_end,
                                  input_size=input_size)

    # return the part of the context, which is already an answer
    return context[score_start_end[1]:score_start_end[2]], score_start_end[0]

### Main Processing Function

Run question answering on specific knowledge base and iterate through the questions.


In [ ]:
def run_question_answering(sources, vocab):
    print(f"Context: {sources}", flush=True)
    context = load_context(sources)

    if len(context) == 0:
        print("Error: Empty context or outside paragraphs")
        return

    while True:
        question = input()
        # if no question - break
        if question == "":
            break

        # measure processing time
        start_time = time.perf_counter()
        answer, score = get_best_answer(
            question=question, context=context, vocab=vocab, input_keys=input_keys
        )
        end_time = time.perf_counter()

        print(f"Question: {question}")
        print(f"Answer: {answer}")
        print(f"Score: {score:.2f}")
        print(f"Time: {end_time - start_time:.2f}s")


## Run

### Run on local paragraphs

Change sources to your own to answer your questions. You can use as many sources as you want. Usually, you need to wait a few seconds for the answer, but the longer context the longer the waiting time. The model is very limited and sensitive for the input. The answer can depend on whether there is a question mark at the end. The model will try to answer any of your questions even there is no good answer in the context, so in that case, you can see random results.

Sample source: Computational complexity theory paragraph (from [here](https://rajpurkar.github.io/SQuAD-explorer/explore/v2.0/dev/Computational_complexity_theory.html))

Sample questions:

- What is the term for a task that generally lends itself to being solved by a computer?
- By what main attribute are computational problems classified utilizing computational complexity theory?
- What branch of theoretical computer science deals with broadly classifying computational problems by difficulty and class of relationship?

If you want to stop the processing just put an empty string.

_Note: Firstly, run the code below and then put your questions in the box._


In [ ]:
sources = [
    "Computational complexity theory is a branch of the theory of computation in theoretical computer "
    "science that focuses on classifying computational problems according to their inherent difficulty, "
    "and relating those classes to each other. A computational problem is understood to be a task that "
    "is in principle amenable to being solved by a computer, which is equivalent to stating that the "
    "problem may be solved by mechanical application of mathematical steps, such as an algorithm."
]

run_question_answering(sources, vocab)


### Run on websites

You can also provide urls. Note that the context (knowledge base) is built from website paragraphs. If some information is outside the paragraphs, the algorithm won't able to find it.

Sample source: [OpenVINO wiki](https://en.wikipedia.org/wiki/OpenVINO)

Sample questions:

- What does OpenVINO mean?
- What is the license for OpenVINO?
- Where can you deploy OpenVINO code?

If you want to stop the processing just put an empty string.

_Note: Firstly, run the code below and then put your questions in the box._


In [ ]:
sources = ["https://en.wikipedia.org/wiki/OpenVINO"]

run_question_answering(sources, vocab)
